In [2]:
import numpy as np
import nibabel as nib
import stitchSurfs as stitch

In [ ]:
lbl = nib.load("/data/mica1/01_programs/micapipe-v0.2.0/parcellations/schaefer-100_conte69_lh.label.gii")
white = nib.load("refsurfs/avg_L_fs_white.surf.gii")
pial = nib.load("refsurfs/avg_L_fs_pial.surf.gii")
idx = stitch.vertices_within_threshold(white, pial, threshold_mm=0.5)
print(len(idx))

white_nomed = stitch.gifti_remove_bad_vertices(white, idx)
# nib.save(white_nomed, "refsurfs/avg_L_fs_white_nomed.surf.gii")
pial_nomed = stitch.gifti_remove_bad_vertices(pial, idx)
# nib.save(pial_nomed, "refsurfs/avg_L_fs_pial_nomed.surf.gii")

hipp_o = nib.load("refsurfs/avg_L_hu_outer.surf.gii")
hipp_i = nib.load("refsurfs/avg_L_hu_inner.surf.gii")
idx = stitch.carve_neocortex_by_distance_gifti(hipp_o,white_nomed)
print(len(idx))

white_nohipp = stitch.gifti_remove_bad_vertices(white_nomed, idx)
nib.save(white_nohipp, "refsurfs/avg_L_fs_white_nohipp.surf.gii")
pial_nohipp = stitch.gifti_remove_bad_vertices(pial_nomed, idx)
nib.save(pial_nohipp, "refsurfs/avg_L_fs_pial_nohipp.surf.gii")

# space-unfold, hipp_midthickness
unfold = nib.load("/data/mica3/BIDS_PNI/derivatives/hippunfold_v1.3.0/hippunfold/sub-PNC001/ses-01/surf/sub-PNC001_ses-01_hemi-L_space-unfold_den-0p5mm_label-hipp_midthickness.surf.gii")
unfold_V = np.asarray([da.data for da in unfold.darrays if da.intent == nib.intent_codes['NIFTI_INTENT_POINTSET']][0])
print(np.min(unfold_V,axis=0))
print(np.max(unfold_V,axis=0))
bridgehead = np.where(unfold_V[:,1] < -199.7)[0]
len(bridgehead)

stitched_surf = stitch.stitch_hippocampus_neocortex_gifti_nearestDP(hipp_o, white_nohipp, bridgehead)
nib.save(stitched_surf, "refsurfs/avg_L_stitched_white.surf.gii")


In [ ]:

template = stitch.make_overlap_stitch_template(white, hipp_o, stitched_surf)
new_stitched = stitch.apply_overlap_stitch_template(pial, hipp_i, template)
nib.save(new_stitched, "refsurfs/avg_L_stitched_pial.surf.gii")

hipp_o = nib.load("refsurfs/avg_R_hu_outer.surf.gii")
white = nib.load("refsurfs/avg_R_fs_white.surf.gii")
new_stitched_white = stitch.apply_overlap_stitch_template(white, hipp_o, template)
nib.save(new_stitched_white, "refsurfs/avg_R_stitched_white.surf.gii")

hipp_i = nib.load("refsurfs/avg_R_hu_inner.surf.gii")
pial = nib.load("refsurfs/avg_R_fs_pial.surf.gii")
new_stitched_pial = stitch.apply_overlap_stitch_template(pial, hipp_i, template)
nib.save(new_stitched_pial, "refsurfs/avg_R_stitched_pial.surf.gii")